In [43]:
import networkx as nx
import numpy as np
import pickle 
from tqdm import tqdm 

In [29]:
data = 'environment'
path = "../gs_taxo/EN/" + str(data) + "_eurovoc_en.taxo"
G = nx.DiGraph()

with open(path, "r") as f:
    for line in f:
        idx, hypo, hyper = line.split("\t")
        hyper = hyper.replace("\n", "")
        G.add_node(hypo)
        G.add_node(hyper)
        G.add_edge(hyper, hypo)

root = data
all_verteces = list(G.nodes)
all_verteces.remove(root)

In [40]:
def clean_dict(pairs, use_lemma, reverse):
    new_pairs = {}
    for key, val in pairs.items():
        if use_lemma:
            term = key[0].split("(")[0].strip()
        else:
            term = key[0]
        target = key[1].split(",")[0]
        new_key = (target, term) if reverse else (term, target)
        new_pairs[new_key] = val

    return new_pairs

in_name = '../data/env/eurovoc/pred_hypernym/lemmas_ppl_clear.pickle'
lemma = True
reverse = True
with open(in_name, "rb") as f:
    ppls = pickle.load(f)

ppls_pairs = clean_dict(ppls, use_lemma=lemma, reverse=reverse)

in_name = '../data/env/eurovoc/leaf_no_leafs/lemmas_ppl_clear.pickle'
lemma = True
reverse = False
with open(in_name, "rb") as f:
    ppls = pickle.load(f)

ppls_hypo = clean_dict(ppls, use_lemma=lemma, reverse=reverse)

In [41]:
def get_graph(G, ppl_pairs):
    nodes = list(G.nodes())
    S = nx.DiGraph()
    for node in nodes:
        candidates = {}
        for key, val in ppl_pairs.items():
            u, v = key
            if v == node:
                candidates[u] = val

        best_node, best_val = min(candidates.items(), key=lambda x: x[1])
        S.add_edge(best_node, node, weight=best_val)
    return S

def conflict_refinment_ppl(G_pred, ppls_hypo, ppls_pairs, conflict_thr):
    edges = list(G_pred.edges())
    for u, v in edges:
        diff = abs(ppls_pairs[(u, v)] - ppls_hypo[(u, v)])

        if diff > conflict_thr:
            # print(diff)
            G_pred.remove_edge(u, v)

def filter_by_child(G_pred, ppls_hypo, thr):
    edges = list(G_pred.edges())
    for u, v in edges:
        if ppls_hypo[(u,v)] < thr:
            G_pred.remove_edge(u,v)
            

In [44]:

Fs = []
thrs = np.arange(1, 150, 1)

for thr in tqdm(thrs):
    G_pred = get_graph(G, ppls_pairs)
    conflict_refinment_ppl(G_pred, ppls_hypo, ppls_pairs, thr)

    P = len(set(G.edges()) & set(G_pred.edges())) / (
    len(set(G_pred.edges())) + 1e-15
    )
    R = len(set(G.edges()) & set(G_pred.edges())) / len(set(G.edges()))
    F = (2 * P * R) / (P + R + 1e-15)

    Fs.append(F)


print(max(Fs))

  0%|          | 0/149 [00:00<?, ?it/s]

100%|██████████| 149/149 [02:59<00:00,  1.21s/it]


TypeError: 'float' object is not iterable

In [45]:
print(max(Fs))

0.4225941422594137


In [46]:
Fs = []
thrs = np.arange(1, 150, 1)

for thr in tqdm(thrs):
    G_pred = get_graph(G, ppls_pairs)
    filter_by_child(G_pred, ppls_hypo, thr)

    P = len(set(G.edges()) & set(G_pred.edges())) / (
    len(set(G_pred.edges())) + 1e-15
    )
    R = len(set(G.edges()) & set(G_pred.edges())) / len(set(G.edges()))
    F = (2 * P * R) / (P + R + 1e-15)

    Fs.append(F)


print(max(Fs))

  1%|▏         | 2/149 [00:03<04:28,  1.82s/it]


RuntimeError: dictionary changed size during iteration

In [20]:
len(set(G.edges()))

261